In [1]:
import zipfile
import os

# Provide the path to your ZIP file in Google Drive
zip_path = '/content/drive/MyDrive/zip-folder.zip'

# Specify the directory where you want to extract the files
extract_path = '/content/extracted_files'

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Files extracted to {extract_path}")

Files extracted to /content/extracted_files


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [3]:
# Define paths to your dataset
train_dir = "extracted_files/train"
val_dir = "extracted_files/val"
test_dir = "extracted_files/test"

In [4]:
# Set parameters
img_height, img_width = 299, 299  # InceptionV3 input size
batch_size = 32

In [5]:
# Load datasets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='int'
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='int'
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='int'
)

Found 6953 files belonging to 100 classes.
Found 1966 files belonging to 100 classes.
Found 1034 files belonging to 100 classes.


In [6]:
# Auto-detect the number of classes
class_names = train_ds.class_names
num_classes = len(class_names)
print(f"Number of classes: {num_classes}")

Number of classes: 100


In [7]:
from tensorflow.keras.applications.inception_v3 import preprocess_input

In [8]:
# Preprocess datasets
train_ds = train_ds.map(lambda x, y: (preprocess_input(x), y))
val_ds = val_ds.map(lambda x, y: (preprocess_input(x), y))
test_ds = test_ds.map(lambda x, y: (preprocess_input(x), y))

In [9]:
# Define the InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False  # Freeze the base model

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [10]:
from tensorflow.keras import models, layers

In [11]:
# Add custom layers on top
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

In [12]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [13]:
# Train the model
epochs = 10
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 78s 256ms/step - accuracy: 0.0478 - loss: 4.5129 - val_accuracy: 0.3591 - val_loss: 3.5053
Epoch 2/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 30s 139ms/step - accuracy: 0.2795 - loss: 3.3280 - val_accuracy: 0.5799 - val_loss: 2.4769
Epoch 3/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 45s 158ms/step - accuracy: 0.4836 - loss: 2.3804 - val_accuracy: 0.7401 - val_loss: 1.7887
Epoch 4/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 41s 159ms/step - accuracy: 0.6380 - loss: 1.7690 - val_accuracy: 0.8250 - val_loss: 1.3314
Epoch 5/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 37s 142ms/step - accuracy: 0.7516 - loss: 1.3268 - val_accuracy: 0.8835 - val_loss: 1.0014
Epoch 6/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 31s 143ms/step - accuracy: 0.8259 - loss: 1.0100 - val_accuracy: 0.9232 - val_loss: 0.7607
Epoch 7/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 41s 144ms/step - accuracy: 0.8803 - loss: 0.7827 - val_accuracy: 0.9527 - val_loss: 0.5861
Epoch 8/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 35s 162ms/step - accuracy: 0.9088 - loss: 0

In [14]:
# Save the entire model
model.save('my_model.h5')

In [15]:
# Train the model for more 40 epochs
epochs = 40
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/40
218/218 ━━━━━━━━━━━━━━━━━━━━ 30s 137ms/step - accuracy: 0.9741 - loss: 0.3172 - val_accuracy: 0.9924 - val_loss: 0.2200
Epoch 2/40
218/218 ━━━━━━━━━━━━━━━━━━━━ 45s 156ms/step - accuracy: 0.9770 - loss: 0.2659 - val_accuracy: 0.9964 - val_loss: 0.1772
Epoch 3/40
218/218 ━━━━━━━━━━━━━━━━━━━━ 31s 141ms/step - accuracy: 0.9833 - loss: 0.2163 - val_accuracy: 0.9975 - val_loss: 0.1415
Epoch 4/40
218/218 ━━━━━━━━━━━━━━━━━━━━ 31s 142ms/step - accuracy: 0.9917 - loss: 0.1763 - val_accuracy: 0.9969 - val_loss: 0.1210
Epoch 5/40
218/218 ━━━━━━━━━━━━━━━━━━━━ 31s 143ms/step - accuracy: 0.9892 - loss: 0.1580 - val_accuracy: 0.9985 - val_loss: 0.0993
Epoch 6/40
218/218 ━━━━━━━━━━━━━━━━━━━━ 41s 143ms/step - accuracy: 0.9937 - loss: 0.1307 - val_accuracy: 0.9995 - val_loss: 0.0828
Epoch 7/40
218/218 ━━━━━━━━━━━━━━━━━━━━ 31s 144ms/step - accuracy: 0.9959 - loss: 0.1131 - val_accuracy: 0.9990 - val_loss: 0.0712
Epoch 8/40
218/218 ━━━━━━━━━━━━━━━━━━━━ 41s 144ms/step - accuracy: 0.9969 - loss: 0

In [16]:
# Evaluate on training data
train_loss, train_acc = model.evaluate(train_ds)
print(f"Train Loss: {train_loss:.4f}\tTrain Accuracy: {train_acc * 100:.2f}%")

# Evaluate on test data
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Loss: {test_loss:.4f}\tTest Accuracy: {test_acc * 100:.2f}%")

218/218 ━━━━━━━━━━━━━━━━━━━━ 28s 128ms/step - accuracy: 1.0000 - loss: 4.9384e-04
Train Loss: 0.0005	Train Accuracy: 100.00%
33/33 ━━━━━━━━━━━━━━━━━━━━ 11s 352ms/step - accuracy: 1.0000 - loss: 0.0021
Test Loss: 0.0025	Test Accuracy: 100.00%


In [18]:
# Save the entire model
model.save('my_model.h5')